In [ ]:
import torch
import torchvision
from torchvision import transforms as T

from PIL import Image, ImageDraw
import cv2
import numpy as np

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
!wget "http://images.cocodataset.org/val2017/000000037777.jpg"

--2024-11-08 12:08:22--  http://images.cocodataset.org/val2017/000000037777.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.30.1, 52.216.54.137, 52.217.226.97, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.30.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40833 (40K) [image/jpeg]
Saving to: ‘000000037777.jpg.2’

000000037777.jpg.2  100%[===================>]  39.88K  --.-KB/s    in 0.03s   

2024-11-08 12:08:22 (1.31 MB/s) - ‘000000037777.jpg.2’ saved [40833/40833]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ig = Image.open('/content/drive/My Drive/it_clinika/content/000000037777.jpg')

In [ ]:
transform = T.ToTensor()
img = transform(ig)

In [ ]:
with torch.no_grad():
  pred = model([img])

In [ ]:
pred
# видим координаты прямоугольников, labels и уверенность сети

[{'boxes': tensor([[291.8007,  75.4907, 351.0379, 229.7333],
          [137.4995, 126.4641, 196.7451, 192.7782],
          [233.8998, 187.4761, 257.3713, 206.0314],
          [ 90.1806, 183.4493, 288.7284, 227.4105],
          [215.5326, 186.7111, 232.5018, 201.2200],
          [229.1946, 178.4547, 242.4028, 189.1366],
          [204.3889, 187.5528, 220.3058, 207.7724],
          [231.4346, 200.6802, 247.8696, 217.5509],
          [217.1698, 200.3502, 231.5009, 214.5559],
          [204.0359, 181.5956, 255.1724, 215.4856],
          [ 98.9669, 120.1149, 110.5809, 174.5164],
          [102.2467, 119.0014, 109.8358, 135.8731],
          [264.9863, 133.9124, 294.1249, 137.5284],
          [232.7792, 188.8490, 246.7510, 200.8903],
          [256.4788, 132.5554, 295.4839, 138.3106],
          [175.0202, 110.8700, 185.6927, 137.3661],
          [273.3569, 135.1965, 294.8690, 137.8414],
          [263.4073, 128.6209, 298.7946, 137.3732],
          [197.2355, 181.2733, 263.8224, 219.3763],
   

In [ ]:
bboxes, labels, scores = pred[0]['boxes'], pred[0]['labels'], pred[0]['scores']

In [ ]:
num = torch.argwhere(scores > 0.9).shape[0]

In [ ]:
num

6

In [ ]:
coco_names = ["person" , "bicycle" , "car" , "motorcycle" , "airplane" , "bus" , "train" , "truck" , "boat" , "traffic light" , "fire hydrant" , "street sign" , "stop sign" , "parking meter" , "bench" , "bird" , "cat" , "dog" , "horse" , "sheep" , "cow" , "elephant" , "bear" , "zebra" , "giraffe" , "hat" , "backpack" , "umbrella" , "shoe" , "eye glasses" , "handbag" , "tie" , "suitcase" ,
"frisbee" , "skis" , "snowboard" , "sports ball" , "kite" , "baseball bat" ,
"baseball glove" , "skateboard" , "surfboard" , "tennis racket" , "bottle" ,
"plate" , "wine glass" , "cup" , "fork" , "knife" , "spoon" , "bowl" ,
"banana" , "apple" , "sandwich" , "orange" , "broccoli" , "carrot" , "hot dog" ,
"pizza" , "donut" , "cake" , "chair" , "couch" , "potted plant" , "bed" ,
"mirror" , "dining table" , "window" , "desk" , "toilet" , "door" , "tv" ,
"laptop" , "mouse" , "remote" , "keyboard" , "cell phone" , "microwave" ,
"oven" , "toaster" , "sink" , "refrigerator" , "blender" , "book" ,
"clock" , "vase" , "scissors" , "teddy bear" , "hair drier" , "toothbrush" , "hair brush"]

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
image = Image.open("/content/drive/My Drive/it_clinika/content/000000037777.jpg")
draw = ImageDraw.Draw(image)

for i in range(num):
  x1,y1,x2,y2 = bboxes[i].numpy().astype('int')

  left_top, right_bottom = (x1, y1), (x2, y2)
  draw.rectangle([left_top, right_bottom], outline='red', width=3)


# Работа с кастомным набором данных

In [ ]:
import torchvision

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

In [ ]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

### train data

In [ ]:
# уберем последний слой и заменим на наш
import numpy
import pandas as pd
import os

In [ ]:
import random
from PIL import Image, ImageDraw
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
train = pd.read_csv('/content/drive/My Drive/it_clinika/dataset/detection/train/bbox.csv')

In [ ]:
train.head()

,Unnamed: 0,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,20,0622cd29e4e0e4f198abf15614819ae8,Cardiomegaly,3,R10,1044.0,1579.0,2098.0,1971.0
1,43,4016b176eaf40d514b0559a0aeb48f21,Cardiomegaly,3,R10,1109.0,1552.0,2416.0,2033.0
2,45,25f2c7b53a6ed09a9aaf73c30357aaf6,Cardiomegaly,3,R8,707.0,1316.0,2026.0,1686.0
3,47,18ee9ef3baea468de2087e0edd85e919,Cardiomegaly,3,R8,862.0,1829.0,1963.0,2185.0
4,133,33403064ce25caa5fda270e6158c6b03,Cardiomegaly,3,R8,1248.0,1392.0,2458.0,1798.0


In [ ]:
df = train

In [ ]:
df.head() # имя, координаты

,Unnamed: 0,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,20,0622cd29e4e0e4f198abf15614819ae8,Cardiomegaly,3,R10,1044.0,1579.0,2098.0,1971.0
1,43,4016b176eaf40d514b0559a0aeb48f21,Cardiomegaly,3,R10,1109.0,1552.0,2416.0,2033.0
2,45,25f2c7b53a6ed09a9aaf73c30357aaf6,Cardiomegaly,3,R8,707.0,1316.0,2026.0,1686.0
3,47,18ee9ef3baea468de2087e0edd85e919,Cardiomegaly,3,R8,862.0,1829.0,1963.0,2185.0
4,133,33403064ce25caa5fda270e6158c6b03,Cardiomegaly,3,R8,1248.0,1392.0,2458.0,1798.0


In [ ]:
unique_imgs = df.image_id.unique()
unique_imgs

array(['0622cd29e4e0e4f198abf15614819ae8',
       '4016b176eaf40d514b0559a0aeb48f21',
       '25f2c7b53a6ed09a9aaf73c30357aaf6',
       '18ee9ef3baea468de2087e0edd85e919',
       '33403064ce25caa5fda270e6158c6b03',
       '389f167c16b70e1eb1aa219fedc373b7',
       '1302aab3d9d19f6bcb9db728e3ce6306',
       '1c1ef26e3b3323f74041f6dd2371cd24',
       '3c98c90840a9290511ae8192107f8250',
       '511e8b50c9a7ecee4e1a8621f262e9ad',
       '21992194880ed480bd0232be383b8268',
       '2c475d89c90a50567ba0cf5ddd62f68a',
       '4d01d09027d1d1e0513de4c8b4fc20e1',
       '2e14a1d545fe84fb87891640ba990781',
       '363dc405e14ed95659d88707f54730de',
       '26be9cde1a3d1a38f19c6753cc9ed40d',
       '2b5b141ce31a62f9b997052a3ca73128',
       '1912296cce436028085697480121b4eb',
       '1720ee54631aff23784053fe1719dfdb',
       '317507bb22f6c10f4f08e8666c0f6ff8',
       '54d7976f3b2da92eafd691971f78ea2b',
       '1fcceea02142af619405d89b9ee156fc',
       '41c40fd3bd0719e48290cd6f3485ca77',
       '507

In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.8 MB/s eta 0:00:00


In [ ]:
import pydicom

In [ ]:
class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(16, 16)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    def __call__(self, img):

        img = np.array(img)

        # переводим изображение в другой формат (L - яркость, A, B - цветовые каналы)
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)

        # изменяем яркость изображения и создаём новое
        l = self.clahe.apply(l)
        lab = cv2.merge((l, a, b))

        # перевод в исходный формат
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        return Image.fromarray(img)

In [ ]:
class CustDat(torch.utils.data.Dataset):
  def __init__(self, df, unique_imgs, indices):
    self.df = df
    self.unique_imgs = unique_imgs
    self.indices = indices
    self.clahe = CLAHETransform()
  def __len__(self):
    return len(self.indices)
  def __getitem__ (self, idx):
    image_name = self.unique_imgs[self.indices[idx]]
    boxes = self.df[self.df.image_id == image_name].values[:,(-4):].astype('float')
    dicom_path = ''
    if os.path.isfile('/content/drive/My Drive/it_clinika/dataset/detection/train/Cardiomegaly/' + image_name + '.dicom'):
      dicom_path = '/content/drive/My Drive/it_clinika/dataset/detection/train/Cardiomegaly/' + image_name + '.dicom'
    elif os.path.isfile('/content/drive/My Drive/it_clinika/dataset/detection/train/Lung Opacity/' + image_name + '.dicom'):
      dicom_path = '/content/drive/My Drive/it_clinika/dataset/detection/train/Lung Opacity/' + image_name + '.dicom'
    else:
      dicom_path = '/content/drive/My Drive/it_clinika/dataset/detection/train/No finding/' + image_name + '.dicom'
    dicom_file = pydicom.dcmread(dicom_path)
    img_data = dicom_file.pixel_array
    img = Image.fromarray(img_data)
    labels = torch.ones((boxes.shape[0]), dtype = torch.int64)
    target = {}
    target['boxes'] = torch.tensor(boxes)
    target['label'] = labels
    return T.ToTensor()(img), target


In [ ]:
train_inds, val_inds = train_test_split(range(unique_imgs.shape[0]), test_size = 0.1)

In [ ]:
def custom_collate(data):
  return data

In [ ]:
train_dl = torch.utils.data.DataLoader(CustDat(df, unique_imgs, train_inds),
                                       batch_size = 16,
                                       shuffle = True,
                                       collate_fn = custom_collate,
                                       pin_memory = True if torch.cuda.is_available() else False)

val_dl = torch.utils.data.DataLoader(CustDat(df, unique_imgs, val_inds),
                                       batch_size = 8,
                                       shuffle = True,
                                       collate_fn = custom_collate,
                                       pin_memory = True if torch.cuda.is_available() else False)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
optimazer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9, weight_decay= 0.0005)
num_epochs = 5

In [ ]:
model.to(device)

for epoch in range(num_epochs):
    epoch_loss = 0
    for data in train_dl:
        imgs = []
        targets = []

        for d in data:
            imgs.append(d[0].to(device).float() / 255.0)

            targ = {
                'boxes': d[1]['boxes'].to(device),
                'labels': d[1]['label'].to(device)
            }
            targets.append(targ)

        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())

        with torch.no_grad():
            epoch_loss += loss.cpu().detach().numpy()

        optimazer.zero_grad()
        loss.backward()
        optimazer.step()

        del imgs, targets, loss, loss_dict
        torch.cuda.empty_cache()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")


NameError: name 'model' is not defined

In [ ]:
model.eval()
data = iter(val_dl).__next__()

In [ ]:
img = data[0][0]
boxes = data[0][1]['boxes']
labels = data[0][1]['label']

In [ ]:
output = model([img.to(device)])
output

In [ ]:
out_bbox = output[0]['boxer']
out_scores = output[0]['scores']

In [ ]:
keep = torchvision.ops.nms(out_bbox, out_scores, 0.45)

In [ ]:
out_bbox.shape, keep.shape

In [ ]:
im = (img.permute(1,2,0).cpu().detach().numpy() * 255).astype('uint8')
im

In [ ]:
vsample = Image.fromarray(im)

In [ ]:
draw = ImageDraw.Draw(vsample)
for box in boxes:
  draw.rectangle(list(box), fill = None, outline = 'red')
vsample